### Biblioteca Numpy
Numpy adiciona a linguagem Python facilidades para a manipulação e cálculos em arrays de grande dimensão. Ela adiciona o tipo de dado "ndarray", tipo que a epmwebapi utiliza para o retorno de séries temporais do EPM Server.

**Conexão ao EPM Server e busca de dados**

In [2]:
import epmwebapi as epm
import numpy as np
import datetime

#Método para não expor usuário e senha do EPM no código fonte
import getpass
user = input('EPM user:')
password = getpass.getpass("EPM password:")

#crie o objeto de conexão informando os endereços do EPM Webserver(Authentication Port e WEB API Port), usuário e senha.
connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', user, password)

bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(hours=1)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

data = bv[bvname].historyReadRaw(queryPeriod)

#Fechando a conexão
connection.close()

EPM user:sa
EPM password:········


In [4]:
# O retorno das funções de leitura da epmwebapi é um ndarray com três campos: Timestamp, Value e Quality.
print(data)

[(24.9, datetime.datetime(2018, 10, 26, 11, 2, 8, tzinfo=datetime.timezone.utc), 0)]


**Índices**

Os índices começam em zero. E podem ser usados indices negativos, começando em -1, para percorrer o ndarray de trás para frente.

In [5]:
# Para acessar os campos use colchetes[]:
data['Value']

array([24.9], dtype=float32)

In [14]:
#acessando o primeiro valor do ndarray 
print(data['Value'][0])
#acessando o terceiro valor
print(data['Value'][2])

#acessando o último valor do ndarray 
print(data['Value'][-1])

#acessando o penúltimo valor do ndarray 
print(data['Value'][-2])

23.7
23.9
23.9
24.0


 **Fatiamentos**
 
 Fatiar é a forma de escolher valores dentro do ndarray. Usamos a sintaxe **ndarray[start : stop : step]**

In [25]:
#imprimindo novamente o array para termos a referência de valores
print(data['Value'])

#acessando os dois primeiros elementos
print(data['Value'][0:2])

#acessando o terceiro e quarto elemento
print(data['Value'][2:4])

#acessando os elementos de índice impar
print(data['Value'][1::2])

[23.7 23.8 23.9 24.  24.1 24.  23.9]
[23.7 23.8]
[23.9 24. ]
[23.8 24.  24. ]


**Cálculos**

O Numpy possui diversos métodos que retornam valores cálculados do array.

In [7]:
# Using NumPy's built-in functions to Find Mean, Median, SD and Variance
mean = np.mean(data['Value'])
median = np.median(data['Value'])
sd = np.std(data['Value'])
variance = np.var(data['Value'])

# Printing the values
print("Mean", mean) 
print("Median", median) 
print("Standard Deviation", sd) 
print("Variance", variance) 

Mean 24.9
Median 24.9
Standard Deviation 0.0
Variance 0.0


### Exemplos práticos

**Remoção de outliers**

Utilizamos a função **percentile** que retorna os quartis, para usar o método descrito aqui: https://www.purplemath.com/modules/boxwhisk3.htm

In [8]:
temperaturas = [ 25, 26, 225, 24, 23, 24, 25, 325, 28, 27]

def removeoutlier(values):
    fator = 1.5
    q3, q1 = np.percentile(values, [75, 25])

    iqr = q3 - q1

    lowpass = q1 - (iqr * fator)

    highpass = q3 + (iqr * fator)
    
    return [v for v in values if v > lowpass and v < highpass]

data = removeoutlier(temperaturas)
print(data)

[25, 26, 24, 23, 24, 25, 28, 27]


**Para mais informações acesse a documentação da biblioteca**

https://docs.scipy.org/doc/numpy/reference/routines.html
